#Algorithme de Berstein-Vazirani

Importations de librairies

In [31]:
from qiskit import QuantumCircuit
import numpy as np

Fonctions

In [32]:
def decimal_to_bit_string(decimal):

    bits_string = ""

    print(np.math.log2(decimal))
    for i in range(round(np.math.log2(decimal)), 0, -1):
        if ((decimal - 2**i) < 0):
            bits_string = bits_string + str(0)
        else:
            bits_string = bits_string + str(1)

    return bits_string

Fonction de création des oracles

In [33]:
def OracleBersteinVazirani(decimal, num_qubits=0):

    bits_string = decimal_to_bit_string(decimal)
    string_length = len(bits_string)

    qc = QuantumCircuit(string_length + 1, string_length)

    string_length = len(bits_string)

    for i in range(string_length):
        if bits_string[i] == '1':
            qc.cx(i, string_length)

    return qc

Fonction pour le circuit de Berstein-Vazirani

In [34]:
def BersteinVazirani(decimal):

    bits_string = decimal_to_bit_string(decimal)
    string_length = len(bits_string)

    qc = QuantumCircuit(string_length + 1, string_length)

    for i in range(string_length):
        qc.h(i)

    qc.append(OracleBersteinVazirani(decimal), range(0, string_length+1))

    for i in range(string_length):
        qc.h(i)

    return qc

Éxecution du circuit

In [35]:
decimals = [np.random.randint(0,50) for i in range(10)]
bit_string_result = []

for decimal in decimals:

    Bernstein_vazirani_circuit = BersteinVazirani(decimal)
    Bernstein_vazirani_circuit.measure(range(Bernstein_vazirani_circuit.num_qubits-1),
                                       range(Bernstein_vazirani_circuit.num_clbits))

    current_bit_string = []

    for i in range (Bernstein_vazirani_circuit.num_clbits):
        current_bit_string.append(Bernstein_vazirani_circuit.clbits[i])

    bit_string_result.append(current_bit_string)

for i in range(len(bit_string_result)):
    print(decimals[i])
    print(bit_string_result[i])

ValueError: math domain error